## For reference paper

In [1]:
# Assuming the FNA file is named "input.fna"
with open("Chinese_hamster_CDS.fna", "r") as fna_file:
    lines = fna_file.readlines()

with open("output.fasta", "w") as fasta_file:
    for i in range(len(lines)):
        if lines[i].startswith(">"):
            fasta_file.write(lines[i])
        else:
            fasta_file.write(lines[i].strip())

In [2]:
pip install biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 31.5 MB/s eta 0:00:00


In [10]:
from collections import Counter

# Custom FASTA parser to handle .fna files with incorrect ">" usage
def parse_fasta_with_validation(file_path):
    sequences = {}
    current_header = None
    current_sequence = []
    valid_nucleotides = {'A', 'T', 'G', 'C'}

    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):  # Possible header line
                # Check if the current sequence is valid and save it
                if current_header and current_sequence:
                    sequences[current_header] = "".join(current_sequence)
                # Start a new sequence
                current_header = line.split()[0]  # Take only the identifier
                current_sequence = []
            else:
                # Check if the line contains valid nucleotide sequences
                if set(line.upper()).issubset(valid_nucleotides | {'N'}):
                    current_sequence.append(line)

        # Save the last sequence
        if current_header and current_sequence:
            sequences[current_header] = "".join(current_sequence)

    return sequences

# Codon-to-amino-acid mapping
codon_table = {
    'A': ['GCT', 'GCC', 'GCA', 'GCG'],
    'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
    'N': ['AAT', 'AAC'],
    'D': ['GAT', 'GAC'],
    'C': ['TGT', 'TGC'],
    'Q': ['CAA', 'CAG'],
    'E': ['GAA', 'GAG'],
    'G': ['GGT', 'GGC', 'GGA', 'GGG'],
    'H': ['CAT', 'CAC'],
    'I': ['ATT', 'ATC', 'ATA'],
    'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
    'K': ['AAA', 'AAG'],
    'M': ['ATG'],
    'F': ['TTT', 'TTC'],
    'P': ['CCT', 'CCC', 'CCA', 'CCG'],
    'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
    'T': ['ACT', 'ACC', 'ACA', 'ACG'],
    'W': ['TGG'],
    'Y': ['TAT', 'TAC'],
    'V': ['GTT', 'GTC', 'GTA', 'GTG'],
}

# Count codons
def count_codons(sequences):
    codon_counts = Counter()
    for seq in sequences.values():
        seq = seq.upper()
        for i in range(0, len(seq) - len(seq) % 3, 3):
            codon = seq[i:i+3]
            if set(codon).issubset({'A', 'T', 'G', 'C'}):  # Ignore invalid codons
                codon_counts[codon] += 1
    return codon_counts

# Load and parse sequences from the .fna file
fasta_file_path = "Chinese_hamster_CDS.fna"  # Replace with your file path
sequences = parse_fasta_with_validation(fasta_file_path)

# Print the total number of sequences processed
print(f"Total sequences processed: {len(sequences)}")

# Count codons
codon_counts = count_codons(sequences)

# Determine the most common codon for each amino acid
most_common_codon_frequency = {}
for aa, codons in codon_table.items():
    total_codons_for_aa = sum(codon_counts[codon] for codon in codons if codon in codon_counts)
    if total_codons_for_aa > 0:
        most_common_codon = max(codons, key=lambda codon: codon_counts.get(codon, 0))
        most_common_codon_frequency[aa] = codon_counts.get(most_common_codon, 0) / total_codons_for_aa

# Print results
print("Most common codon frequencies:")
for aa, freq in most_common_codon_frequency.items():
    print(f"{aa}: {freq:.3f}")


Total sequences processed: 47351
Most common codon frequencies:
A: 0.363
R: 0.234
N: 0.524
D: 0.518
C: 0.513
Q: 0.727
E: 0.559
G: 0.308
H: 0.543
I: 0.456
L: 0.369
K: 0.572
M: 1.000
F: 0.526
P: 0.323
S: 0.225
T: 0.323
W: 1.000
Y: 0.540
V: 0.443


In [12]:
from Bio.Seq import Seq
from Bio import SeqIO
from collections import Counter

# Define the codon-to-amino-acid mapping
codon_table = {
    'A': ['GCU', 'GCC', 'GCA', 'GCG'],
    'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
    'N': ['AAU', 'AAC'],
    'D': ['GAU', 'GAC'],
    'C': ['UGU', 'UGC'],
    'Q': ['CAA', 'CAG'],
    'E': ['GAA', 'GAG'],
    'G': ['GGU', 'GGC', 'GGA', 'GGG'],
    'H': ['CAU', 'CAC'],
    'I': ['AUU', 'AUC', 'AUA'],
    'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'],
    'K': ['AAA', 'AAG'],
    'M': ['AUG'],
    'F': ['UUU', 'UUC'],
    'P': ['CCU', 'CCC', 'CCA', 'CCG'],
    'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'],
    'T': ['ACU', 'ACC', 'ACA', 'ACG'],
    'W': ['UGG'],
    'Y': ['UAU', 'UAC'],
    'V': ['GUU', 'GUC', 'GUA', 'GUG'],
}

# Replace codon frequencies with the updated values
most_common_codon_frequency = {
    'A': 0.363, 'R': 0.234, 'N': 0.524, 'D': 0.518, 'C': 0.513,
    'Q': 0.727, 'E': 0.559, 'G': 0.308, 'H': 0.543, 'I': 0.456,
    'L': 0.369, 'K': 0.572, 'M': 1.000, 'F': 0.526, 'P': 0.323,
    'S': 0.225, 'T': 0.323, 'W': 1.000, 'Y': 0.540, 'V': 0.443
}

# Custom parser for .fna files with incorrect ">" usage
def parse_fasta_with_validation(file_path):
    sequences = {}
    current_header = None
    current_sequence = []
    valid_nucleotides = {'A', 'T', 'G', 'C', 'N'}

    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):  # Possible header line
                # Check if the current sequence is valid and save it
                if current_header and current_sequence:
                    sequences[current_header] = "".join(current_sequence)
                # Start a new sequence
                current_header = line.split()[0]  # Keep only the identifier
                current_sequence = []
            else:
                # Check if the line contains valid nucleotide sequences
                if set(line.upper()).issubset(valid_nucleotides):
                    current_sequence.append(line)

        # Save the last sequence
        if current_header and current_sequence:
            sequences[current_header] = "".join(current_sequence)

    return sequences

# Parse the FASTA file and translate sequences into proteins
def calculate_amino_acid_frequencies(fasta_file):
    amino_acid_counts = Counter()
    total_amino_acids = 0

    sequences = parse_fasta_with_validation(fasta_file)  # Use the custom parser

    for seq in sequences.values():
        # Translate nucleotide sequence into protein
        protein_seq = str(Seq(seq).translate(to_stop=True))
        amino_acid_counts.update(protein_seq)
        total_amino_acids += len(protein_seq)

    # Calculate amino acid frequencies
    amino_acid_frequencies = {
        aa: count / total_amino_acids for aa, count in amino_acid_counts.items()
    }

    return amino_acid_frequencies

# Calculate theoretical maximum accuracy
def calculate_theoretical_maximum_accuracy(fasta_file):
    # Get amino acid frequencies
    amino_acid_frequencies = calculate_amino_acid_frequencies(fasta_file)

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = sum(
        amino_acid_frequencies.get(aa, 0) * most_common_codon_frequency.get(aa, 0)
        for aa in most_common_codon_frequency
    )

    return theoretical_accuracy

# Input FASTA file
fasta_file_path = "Chinese_hamster_CDS.fna"  # Replace with your FASTA file path

# Calculate and print results
theoretical_accuracy = calculate_theoretical_maximum_accuracy(fasta_file_path)
print(f"Theoretical Maximum Accuracy: {theoretical_accuracy:.4f}")


Theoretical Maximum Accuracy: 0.4422


##Brain dataset

In [15]:
from collections import Counter

# Codon-to-amino-acid mapping
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
}

# Function to combine FASTA files
def combine_fasta_files(input_files, output_file):
    with open(output_file, 'w') as outfile:
        for fname in input_files:
            with open(fname) as infile:
                for line in infile:
                    if line.strip():  # Ignore blank lines
                        outfile.write(line)

# Function to preprocess sequences and remove invalid characters
def preprocess_fasta(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        sequence = ''
        for line in infile:
            line = line.strip()
            if line.startswith('>'):
                if sequence:
                    outfile.write(sequence + '\n')
                    sequence = ''
                outfile.write(line + '\n')
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))
        if sequence:
            outfile.write(sequence + '\n')

# Function to count codons and amino acids
def count_codons_and_amino_acids(fasta_file):
    codon_counts = Counter()
    amino_acid_counts = Counter()
    total_sequences = 0

    with open(fasta_file, 'r') as f:
        sequence = ''
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if sequence:  # Process the previous sequence
                    total_sequences += 1
                    for i in range(0, len(sequence) - 2, 3):
                        codon = sequence[i:i+3]
                        if len(codon) == 3:
                            codon_counts[codon] += 1
                            amino_acid = codon_table.get(codon, 'X')
                            amino_acid_counts[amino_acid] += 1
                    sequence = ''
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))

        if sequence:  # Count the last sequence if not already counted
            total_sequences += 1
            for i in range(0, len(sequence) - 2, 3):
                codon = sequence[i:i+3]
                if len(codon) == 3:
                    codon_counts[codon] += 1
                    amino_acid = codon_table.get(codon, 'X')
                    amino_acid_counts[amino_acid] += 1

    print(f"Total sequences processed: {total_sequences}")
    return codon_counts, amino_acid_counts

# Function to calculate codon frequencies
def calculate_codon_frequencies(codon_counts):
    codon_frequencies = {}
    for codon, count in codon_counts.items():
        amino_acid = codon_table.get(codon, 'X')
        if amino_acid not in codon_frequencies:
            codon_frequencies[amino_acid] = {}
        codon_frequencies[amino_acid][codon] = count
    return codon_frequencies

# Function to calculate theoretical maximum accuracy
def calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies):
    total_amino_acids = sum(amino_acid_counts.values()) - amino_acid_counts.get('*', 0)  # Exclude stop codons

    # Calculate amino acid frequencies
    amino_acid_frequencies = {
        aa: count / total_amino_acids for aa, count in amino_acid_counts.items() if aa != '*'
    }

    # Determine most common codon frequencies
    most_common_codon_frequency = {}
    for amino_acid, codons in codon_frequencies.items():
        total = sum(codons.values())
        if total > 0:
            most_common_codon = max(codons, key=codons.get)
            most_common_codon_frequency[amino_acid] = codons[most_common_codon] / total

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = sum(
        amino_acid_frequencies.get(aa, 0) * most_common_codon_frequency.get(aa, 0)
        for aa in amino_acid_frequencies
    )

    return theoretical_accuracy

# Main script
if __name__ == "__main__":
    # Input files
    input_files = ['evaluation_brainlist.fasta', 'prediction_brainlist.fasta', 'training_brainlist.fasta']
    combined_fasta = 'combined.fasta'
    preprocessed_fasta = 'preprocessed_combined.fasta'

    # Combine the files
    combine_fasta_files(input_files, combined_fasta)
    print(f"Combined files into {combined_fasta}")

    # Preprocess the combined FASTA file
    preprocess_fasta(combined_fasta, preprocessed_fasta)
    print(f"Preprocessed combined file into {preprocessed_fasta}")

    # Count codons and amino acids
    codon_counts, amino_acid_counts = count_codons_and_amino_acids(preprocessed_fasta)

    # Calculate codon frequencies
    codon_frequencies = calculate_codon_frequencies(codon_counts)

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies)

    # Print results
    print(f"Theoretical Maximum Accuracy: {theoretical_accuracy:.4f}")


Combined files into combined.fasta
Preprocessed combined file into preprocessed_combined.fasta
Total sequences processed: 679
Theoretical Maximum Accuracy: 0.4804


## Muscle dataset

In [16]:
from collections import Counter

# Codon-to-amino-acid mapping
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
}

# Function to combine FASTA files
def combine_fasta_files(input_files, output_file):
    with open(output_file, 'w') as outfile:
        for fname in input_files:
            with open(fname) as infile:
                for line in infile:
                    if line.strip():  # Ignore blank lines
                        outfile.write(line)

# Function to preprocess sequences and remove invalid characters
def preprocess_fasta(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        sequence = ''
        for line in infile:
            line = line.strip()
            if line.startswith('>'):
                if sequence:
                    outfile.write(sequence + '\n')
                    sequence = ''
                outfile.write(line + '\n')
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))
        if sequence:
            outfile.write(sequence + '\n')

# Function to count codons and amino acids
def count_codons_and_amino_acids(fasta_file):
    codon_counts = Counter()
    amino_acid_counts = Counter()
    total_sequences = 0

    with open(fasta_file, 'r') as f:
        sequence = ''
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if sequence:  # Process the previous sequence
                    total_sequences += 1
                    for i in range(0, len(sequence) - 2, 3):
                        codon = sequence[i:i+3]
                        if len(codon) == 3:
                            codon_counts[codon] += 1
                            amino_acid = codon_table.get(codon, 'X')
                            amino_acid_counts[amino_acid] += 1
                    sequence = ''
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))

        if sequence:  # Count the last sequence if not already counted
            total_sequences += 1
            for i in range(0, len(sequence) - 2, 3):
                codon = sequence[i:i+3]
                if len(codon) == 3:
                    codon_counts[codon] += 1
                    amino_acid = codon_table.get(codon, 'X')
                    amino_acid_counts[amino_acid] += 1

    print(f"Total sequences processed: {total_sequences}")
    return codon_counts, amino_acid_counts

# Function to calculate codon frequencies
def calculate_codon_frequencies(codon_counts):
    codon_frequencies = {}
    for codon, count in codon_counts.items():
        amino_acid = codon_table.get(codon, 'X')
        if amino_acid not in codon_frequencies:
            codon_frequencies[amino_acid] = {}
        codon_frequencies[amino_acid][codon] = count
    return codon_frequencies

# Function to calculate theoretical maximum accuracy
def calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies):
    total_amino_acids = sum(amino_acid_counts.values()) - amino_acid_counts.get('*', 0)  # Exclude stop codons

    # Calculate amino acid frequencies
    amino_acid_frequencies = {
        aa: count / total_amino_acids for aa, count in amino_acid_counts.items() if aa != '*'
    }

    # Determine most common codon frequencies
    most_common_codon_frequency = {}
    for amino_acid, codons in codon_frequencies.items():
        total = sum(codons.values())
        if total > 0:
            most_common_codon = max(codons, key=codons.get)
            most_common_codon_frequency[amino_acid] = codons[most_common_codon] / total

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = sum(
        amino_acid_frequencies.get(aa, 0) * most_common_codon_frequency.get(aa, 0)
        for aa in amino_acid_frequencies
    )

    return theoretical_accuracy

# Main script
if __name__ == "__main__":
    # Input files
    input_files = ['evaluation_musclelist.fasta', 'prediction_musclelist.fasta', 'training_musclelist.fasta']
    combined_fasta = 'combined.fasta'
    preprocessed_fasta = 'preprocessed_combined.fasta'

    # Combine the files
    combine_fasta_files(input_files, combined_fasta)
    print(f"Combined files into {combined_fasta}")

    # Preprocess the combined FASTA file
    preprocess_fasta(combined_fasta, preprocessed_fasta)
    print(f"Preprocessed combined file into {preprocessed_fasta}")

    # Count codons and amino acids
    codon_counts, amino_acid_counts = count_codons_and_amino_acids(preprocessed_fasta)

    # Calculate codon frequencies
    codon_frequencies = calculate_codon_frequencies(codon_counts)

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies)

    # Print results
    print(f"Theoretical Maximum Accuracy: {theoretical_accuracy:.4f}")


Combined files into combined.fasta
Preprocessed combined file into preprocessed_combined.fasta
Total sequences processed: 903
Theoretical Maximum Accuracy: 0.4725


## Liver data

In [17]:
from collections import Counter

# Codon-to-amino-acid mapping
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
}

# Function to combine FASTA files
def combine_fasta_files(input_files, output_file):
    with open(output_file, 'w') as outfile:
        for fname in input_files:
            with open(fname) as infile:
                for line in infile:
                    if line.strip():  # Ignore blank lines
                        outfile.write(line)

# Function to preprocess sequences and remove invalid characters
def preprocess_fasta(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        sequence = ''
        for line in infile:
            line = line.strip()
            if line.startswith('>'):
                if sequence:
                    outfile.write(sequence + '\n')
                    sequence = ''
                outfile.write(line + '\n')
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))
        if sequence:
            outfile.write(sequence + '\n')

# Function to count codons and amino acids
def count_codons_and_amino_acids(fasta_file):
    codon_counts = Counter()
    amino_acid_counts = Counter()
    total_sequences = 0

    with open(fasta_file, 'r') as f:
        sequence = ''
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if sequence:  # Process the previous sequence
                    total_sequences += 1
                    for i in range(0, len(sequence) - 2, 3):
                        codon = sequence[i:i+3]
                        if len(codon) == 3:
                            codon_counts[codon] += 1
                            amino_acid = codon_table.get(codon, 'X')
                            amino_acid_counts[amino_acid] += 1
                    sequence = ''
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))

        if sequence:  # Count the last sequence if not already counted
            total_sequences += 1
            for i in range(0, len(sequence) - 2, 3):
                codon = sequence[i:i+3]
                if len(codon) == 3:
                    codon_counts[codon] += 1
                    amino_acid = codon_table.get(codon, 'X')
                    amino_acid_counts[amino_acid] += 1

    print(f"Total sequences processed: {total_sequences}")
    return codon_counts, amino_acid_counts

# Function to calculate codon frequencies
def calculate_codon_frequencies(codon_counts):
    codon_frequencies = {}
    for codon, count in codon_counts.items():
        amino_acid = codon_table.get(codon, 'X')
        if amino_acid not in codon_frequencies:
            codon_frequencies[amino_acid] = {}
        codon_frequencies[amino_acid][codon] = count
    return codon_frequencies

# Function to calculate theoretical maximum accuracy
def calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies):
    total_amino_acids = sum(amino_acid_counts.values()) - amino_acid_counts.get('*', 0)  # Exclude stop codons

    # Calculate amino acid frequencies
    amino_acid_frequencies = {
        aa: count / total_amino_acids for aa, count in amino_acid_counts.items() if aa != '*'
    }

    # Determine most common codon frequencies
    most_common_codon_frequency = {}
    for amino_acid, codons in codon_frequencies.items():
        total = sum(codons.values())
        if total > 0:
            most_common_codon = max(codons, key=codons.get)
            most_common_codon_frequency[amino_acid] = codons[most_common_codon] / total

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = sum(
        amino_acid_frequencies.get(aa, 0) * most_common_codon_frequency.get(aa, 0)
        for aa in amino_acid_frequencies
    )

    return theoretical_accuracy

# Main script
if __name__ == "__main__":
    # Input files
    input_files = ['evaluation_Liverlist.fasta', 'prediction_Liverlist.fasta', 'training_Liverlist.fasta']
    combined_fasta = 'combined.fasta'
    preprocessed_fasta = 'preprocessed_combined.fasta'

    # Combine the files
    combine_fasta_files(input_files, combined_fasta)
    print(f"Combined files into {combined_fasta}")

    # Preprocess the combined FASTA file
    preprocess_fasta(combined_fasta, preprocessed_fasta)
    print(f"Preprocessed combined file into {preprocessed_fasta}")

    # Count codons and amino acids
    codon_counts, amino_acid_counts = count_codons_and_amino_acids(preprocessed_fasta)

    # Calculate codon frequencies
    codon_frequencies = calculate_codon_frequencies(codon_counts)

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies)

    # Print results
    print(f"Theoretical Maximum Accuracy: {theoretical_accuracy:.4f}")


Combined files into combined.fasta
Preprocessed combined file into preprocessed_combined.fasta
Total sequences processed: 870
Theoretical Maximum Accuracy: 0.4808


# TMA for human secretory genes 

In [1]:
from collections import Counter

# Codon-to-amino-acid mapping
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
}

# Function to combine FASTA files
def combine_fasta_files(input_files, output_file):
    with open(output_file, 'w') as outfile:
        for fname in input_files:
            with open(fname) as infile:
                for line in infile:
                    if line.strip():  # Ignore blank lines
                        outfile.write(line)

# Function to preprocess sequences and remove invalid characters
def preprocess_fasta(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        sequence = ''
        for line in infile:
            line = line.strip()
            if line.startswith('>'):
                if sequence:
                    outfile.write(sequence + '\n')
                    sequence = ''
                outfile.write(line + '\n')
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))
        if sequence:
            outfile.write(sequence + '\n')

# Function to count codons and amino acids
def count_codons_and_amino_acids(fasta_file):
    codon_counts = Counter()
    amino_acid_counts = Counter()
    total_sequences = 0

    with open(fasta_file, 'r') as f:
        sequence = ''
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if sequence:  # Process the previous sequence
                    total_sequences += 1
                    for i in range(0, len(sequence) - 2, 3):
                        codon = sequence[i:i+3]
                        if len(codon) == 3:
                            codon_counts[codon] += 1
                            amino_acid = codon_table.get(codon, 'X')
                            amino_acid_counts[amino_acid] += 1
                    sequence = ''
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))

        if sequence:  # Count the last sequence if not already counted
            total_sequences += 1
            for i in range(0, len(sequence) - 2, 3):
                codon = sequence[i:i+3]
                if len(codon) == 3:
                    codon_counts[codon] += 1
                    amino_acid = codon_table.get(codon, 'X')
                    amino_acid_counts[amino_acid] += 1

    print(f"Total sequences processed: {total_sequences}")
    return codon_counts, amino_acid_counts

# Function to calculate codon frequencies
def calculate_codon_frequencies(codon_counts):
    codon_frequencies = {}
    for codon, count in codon_counts.items():
        amino_acid = codon_table.get(codon, 'X')
        if amino_acid not in codon_frequencies:
            codon_frequencies[amino_acid] = {}
        codon_frequencies[amino_acid][codon] = count
    return codon_frequencies

# Function to calculate theoretical maximum accuracy
def calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies):
    total_amino_acids = sum(amino_acid_counts.values()) - amino_acid_counts.get('*', 0)  # Exclude stop codons

    # Calculate amino acid frequencies
    amino_acid_frequencies = {
        aa: count / total_amino_acids for aa, count in amino_acid_counts.items() if aa != '*'
    }

    # Determine most common codon frequencies
    most_common_codon_frequency = {}
    for amino_acid, codons in codon_frequencies.items():
        total = sum(codons.values())
        if total > 0:
            most_common_codon = max(codons, key=codons.get)
            most_common_codon_frequency[amino_acid] = codons[most_common_codon] / total

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = sum(
        amino_acid_frequencies.get(aa, 0) * most_common_codon_frequency.get(aa, 0)
        for aa in amino_acid_frequencies
    )

    return theoretical_accuracy

# Main script
if __name__ == "__main__":
    # Input files
    input_files = ['evaluation_NMhumanseclist.fasta', 'prediction_NMhumanseclist.fasta', 'training_NMhumanseclist.fasta']
    combined_fasta = 'combinedhumsec.fasta'
    preprocessed_fasta = 'preprocessed_humanseccombined.fasta'

    # Combine the files
    combine_fasta_files(input_files, combined_fasta)
    print(f"Combined files into {combined_fasta}")

    # Preprocess the combined FASTA file
    preprocess_fasta(combined_fasta, preprocessed_fasta)
    print(f"Preprocessed combined file into {preprocessed_fasta}")

    # Count codons and amino acids
    codon_counts, amino_acid_counts = count_codons_and_amino_acids(preprocessed_fasta)

    # Calculate codon frequencies
    codon_frequencies = calculate_codon_frequencies(codon_counts)

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies)

    # Print results
    print(f"Theoretical Maximum Accuracy: {theoretical_accuracy:.4f}")


Combined files into combinedhumsec.fasta
Preprocessed combined file into preprocessed_humanseccombined.fasta
Total sequences processed: 1555
Theoretical Maximum Accuracy: 0.4813


# TMA for mouse secretory genes 

In [3]:
from collections import Counter

# Codon-to-amino-acid mapping
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
}

# Function to combine FASTA files
def combine_fasta_files(input_files, output_file):
    with open(output_file, 'w') as outfile:
        for fname in input_files:
            with open(fname) as infile:
                for line in infile:
                    if line.strip():  # Ignore blank lines
                        outfile.write(line)

# Function to preprocess sequences and remove invalid characters
def preprocess_fasta(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        sequence = ''
        for line in infile:
            line = line.strip()
            if line.startswith('>'):
                if sequence:
                    outfile.write(sequence + '\n')
                    sequence = ''
                outfile.write(line + '\n')
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))
        if sequence:
            outfile.write(sequence + '\n')

# Function to count codons and amino acids
def count_codons_and_amino_acids(fasta_file):
    codon_counts = Counter()
    amino_acid_counts = Counter()
    total_sequences = 0

    with open(fasta_file, 'r') as f:
        sequence = ''
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if sequence:  # Process the previous sequence
                    total_sequences += 1
                    for i in range(0, len(sequence) - 2, 3):
                        codon = sequence[i:i+3]
                        if len(codon) == 3:
                            codon_counts[codon] += 1
                            amino_acid = codon_table.get(codon, 'X')
                            amino_acid_counts[amino_acid] += 1
                    sequence = ''
            else:
                sequence += ''.join(filter(lambda x: x in {'A', 'T', 'G', 'C'}, line.upper()))

        if sequence:  # Count the last sequence if not already counted
            total_sequences += 1
            for i in range(0, len(sequence) - 2, 3):
                codon = sequence[i:i+3]
                if len(codon) == 3:
                    codon_counts[codon] += 1
                    amino_acid = codon_table.get(codon, 'X')
                    amino_acid_counts[amino_acid] += 1

    print(f"Total sequences processed: {total_sequences}")
    return codon_counts, amino_acid_counts

# Function to calculate codon frequencies
def calculate_codon_frequencies(codon_counts):
    codon_frequencies = {}
    for codon, count in codon_counts.items():
        amino_acid = codon_table.get(codon, 'X')
        if amino_acid not in codon_frequencies:
            codon_frequencies[amino_acid] = {}
        codon_frequencies[amino_acid][codon] = count
    return codon_frequencies

# Function to calculate theoretical maximum accuracy
def calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies):
    total_amino_acids = sum(amino_acid_counts.values()) - amino_acid_counts.get('*', 0)  # Exclude stop codons

    # Calculate amino acid frequencies
    amino_acid_frequencies = {
        aa: count / total_amino_acids for aa, count in amino_acid_counts.items() if aa != '*'
    }

    # Determine most common codon frequencies
    most_common_codon_frequency = {}
    for amino_acid, codons in codon_frequencies.items():
        total = sum(codons.values())
        if total > 0:
            most_common_codon = max(codons, key=codons.get)
            most_common_codon_frequency[amino_acid] = codons[most_common_codon] / total

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = sum(
        amino_acid_frequencies.get(aa, 0) * most_common_codon_frequency.get(aa, 0)
        for aa in amino_acid_frequencies
    )

    return theoretical_accuracy

# Main script
if __name__ == "__main__":
    # Input files
    input_files = ['evaluation_NMmouseseclist.fasta', 'prediction_NMmouseseclist.fasta', 'training_NMmouseseclist.fasta']
    combined_fasta = 'combinedmousesec.fasta'
    preprocessed_fasta = 'preprocessed_mousecombined.fasta'

    # Combine the files
    combine_fasta_files(input_files, combined_fasta)
    print(f"Combined files into {combined_fasta}")

    # Preprocess the combined FASTA file
    preprocess_fasta(combined_fasta, preprocessed_fasta)
    print(f"Preprocessed combined file into {preprocessed_fasta}")

    # Count codons and amino acids
    codon_counts, amino_acid_counts = count_codons_and_amino_acids(preprocessed_fasta)

    # Calculate codon frequencies
    codon_frequencies = calculate_codon_frequencies(codon_counts)

    # Calculate theoretical maximum accuracy
    theoretical_accuracy = calculate_theoretical_maximum_accuracy(amino_acid_counts, codon_frequencies)

    # Print results
    print(f"Theoretical Maximum Accuracy: {theoretical_accuracy:.4f}")


Combined files into combinedmousesec.fasta
Preprocessed combined file into preprocessed_mousecombined.fasta
Total sequences processed: 1319
Theoretical Maximum Accuracy: 0.4714
